# from final_submission

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import CreatingNewDataframe
from functions.date_split import split_month_test

%matplotlib inline

from sklearn.model_selection import cross_val_score

np.random.seed(seed=42)

from score_submission import our_scorer, scorer
from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df_initial = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_initial.ATM_ID)
np.random.shuffle(ids)

In [42]:
df.DATE[-1]

KeyError: -1

In [40]:
df.DATE.shift(2)

0               NaT
1               NaT
2        2015-01-01
3        2015-01-02
4        2015-01-03
5        2015-01-04
6        2015-01-05
7        2015-01-06
8        2015-01-07
9        2015-01-08
10       2015-01-09
11       2015-01-10
12       2015-01-11
13       2015-01-12
14       2015-01-13
15       2015-01-14
16       2015-01-15
17       2015-01-16
18       2015-01-17
19       2015-01-18
20       2015-01-19
21       2015-01-20
22       2015-01-21
23       2015-01-22
24       2015-01-23
25       2015-01-24
26       2015-01-25
27       2015-01-26
28       2015-01-27
29       2015-01-28
            ...    
287370   2017-07-15
287371   2017-07-16
287372   2017-07-17
287373   2017-07-18
287374   2017-07-19
287375   2017-07-20
287376   2017-07-21
287377   2017-07-22
287378   2017-07-23
287379   2017-07-24
287380   2017-07-25
287381   2017-07-26
287382   2017-07-27
287383   2017-07-28
287384   2017-07-29
287385   2017-07-30
287386   2017-07-31
287387   2017-08-01
287388   2017-08-02


In [3]:
class SeparatingAtmIds(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        self.train = True
        return self

    def transform(self, X):
        if self.train:
            print("TRAIN ONLY CALLED")
            return CreatingNewDataframe.CleanDataFrame_TRAIN_ONLY(df=X, window_size=15)
        else:
            print("TEST ONLY CALLED")
            return CreatingNewDataframe.CleanDataFrame_TEST(df=X, window_size=15)

In [4]:
class DroppingAtmIdsIn_X(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x=None, y=None):
        return self

    def transform(self, X):
        return X 

In [6]:
class FinalStepForSubmission(BaseEstimator, TransformerMixin):
    """
    FINAL
    """
    
    def fit(self, X, y=None):
        return self

    def predict(self, X):
        X = X[['DATE', "ATM_ID", "y_predict"]]
        X.columns = ['DATE', "ATM_ID", "CLIENT_OUT"]
        print(len(np.unique(X.ATM_ID)))
        return X
        
#     def transform(self, X):
#         raise "asdasdasd"

class WrapperForEstimator(BaseEstimator, TransformerMixin):
    """
    FINAL
    """
    def __init__(self, estimator):
        self.estimator = estimator
    
    
    def fit(self, X, y=None):
#         print("fit called")
        self.estimator.fit(X[['ATM_ID']], y)
        return self

    def transform(self, X):
#         print("transform called")
        y_pred = self.estimator.predict(X[['ATM_ID']])
        X_tr = X.copy()
        X_tr['y_predict'] = y_pred
        return X_tr

In [8]:
pipeline = Pipeline([
#     ('SeparatingAtmIds', SeparatingAtmIds()),
    
    #('DroppingAtmIdsIn_X', )
    
    ('lasso', WrapperForEstimator(linear_model.Lasso())), #predict step
    
   
    ("submission", FinalStepForSubmission()),
    
    ]
)

In [9]:
X = df_initial[['DATE', 'ATM_ID']]
y = df_initial[['CLIENT_OUT']]


In [10]:
pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('lasso', WrapperForEstimator(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))), ('submission', FinalStepForSubmission())])

In [11]:
pipeline.predict(X)

300


,DATE,ATM_ID,CLIENT_OUT
0,2015-01-01,0,315602.973975
1,2015-01-02,0,315602.973975
2,2015-01-03,0,315602.973975
3,2015-01-04,0,315602.973975
4,2015-01-05,0,315602.973975
5,2015-01-06,0,315602.973975
6,2015-01-07,0,315602.973975
7,2015-01-08,0,315602.973975
8,2015-01-09,0,315602.973975
9,2015-01-10,0,315602.973975


# My Old code

In [12]:
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib inline
import pandas as pd

In [13]:
from tqdm import tqdm_notebook

In [26]:
df = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
df['week_day'] = df.DATE.dt.dayofweek

In [31]:
gb = df['CLIENT_OUT'].groupby(df['ATM_ID'])

In [36]:
class MeanByIdFeature(BaseEstimator, TransformerMixin):
    def fit(self, X=None, y=None):
        return self

    def transform(self, X):
        return X['CLIENT_OUT'].groupby(X['ATM_ID']).transform('mean')

In [34]:

df['MEAN'] = df['CLIENT_OUT'].groupby(df['ATM_ID']).transform('mean')

In [35]:
df

,DATE,ATM_ID,CLIENT_OUT,week_day,Data4,MEAN
0,2015-01-01,0,91600,3,438624.425887,438624.425887
1,2015-01-02,0,136500,4,438624.425887,438624.425887
2,2015-01-03,0,335400,5,438624.425887,438624.425887
3,2015-01-04,0,379000,6,438624.425887,438624.425887
4,2015-01-05,0,344100,0,438624.425887,438624.425887
5,2015-01-06,0,575300,1,438624.425887,438624.425887
6,2015-01-07,0,158200,2,438624.425887,438624.425887
7,2015-01-08,0,296500,3,438624.425887,438624.425887
8,2015-01-09,0,453400,4,438624.425887,438624.425887
9,2015-01-10,0,415900,5,438624.425887,438624.425887


In [16]:
gb.mean()

,CLIENT_OUT,week_day
ATM_ID,,
0,4.386244e+05,3.001044
2,4.842599e+04,3.001044
3,2.692042e+05,3.001044
4,5.138682e+05,3.001044
5,3.198094e+05,3.001044
6,6.579970e+05,3.001044
7,1.417449e+05,3.001044
8,1.734052e+05,3.001044
9,1.649730e+05,3.001044


In [20]:
df['MEAN'] = gb.transform(np.mean)

ValueError: Wrong number of items passed 2, placement implies 1

In [ ]:
from sklearn.model_selection import train_test_split
ids = df.ATM_ID.unique()

In [ ]:
from functions.date_split import split_month_test
train_df, test_df = split_month_test(df, 33, 100, 10)
train_df.describe(include='all')

In [ ]:
df.describe(include='all')

In [ ]:
df_pivot = pd.pivot_table(df, values='CLIENT_OUT', index='ATM_ID', columns='DATE', margins=True, margins_name='All')

df.CLIENT_OUT.hist(bins=100)

In [ ]:
ids = df.ATM_ID.unique()

for id in ids:
    plt.figure(figsize=(15,10))
    plt.title('ATM_ID {0}'.format(id))
    plt.plot(df[df.ATM_ID == id].DATE, df[df.ATM_ID==id].CLIENT_OUT)
    plt.show()

In [ ]:
# The dates to predict
pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']
rows = []
for id in ids:
    mean = df[df.ATM_ID==id].CLIENT_OUT.mean()
    for date in pred_dates:
        rows.append((date, id, mean))
submission = pd.DataFrame(rows, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])
submission.to_csv('submission.csv', index=False)

In [ ]:
len(pred_dates)

In [ ]:
def mean_submission(df):
    res = []
    for id in df.ATM_ID.unique():
        mean = df[df.ATM_ID==id].CLIENT_OUT.mean()
        for date in df.DATE.unique():
            res.append((date, id, mean))
    res = pd.DataFrame(res, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])
    return res

## Get last month to test

In [ ]:
from datetime import datetime, timedelta
month = 30
d = datetime.today() - timedelta(days=month)

In [ ]:
from score_submission import scorer

In [ ]:
from functions.date_split import split_month_test
train_df, test_df = split_month_test(df, len(pred_dates), 100)

In [ ]:
test_df[test_df.ATM_ID==0]

In [ ]:
test_df.dtypes

In [ ]:
mn = mean_submission(test_df)
mn.dtypes

In [ ]:
scorer(test_df.reset_index(), mean_submission(test_df))

In [ ]:
import pickle

In [ ]:
model = pickle.load(open("rf_clf.pickle", "rb"))

In [ ]:
model.predict(test_df)